## Make Lenet-1 Model and Convert ONNX
- Keras로 Lenet-1 모델 생성 
- 학습은 Mnist Data로 진행
- <b> .h5 모델 로드 후 .onnx로 변환 </b>


In [5]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras import utils
from PIL import Image

# image resize 함수
def img_resize(value, img_array):
    print("--images are being resizing--")
    result = np.zeros((len(img_array), value[0], value[1]))

    for index in range(len(img_array)):
        img = Image.fromarray(img_array[index], 'L')
        img = img.resize((value[0], value[1]))
        img = np.array(img)
        result[index] = img
    print("--image resize complete--")
    return result

# mnist Data Load 및
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

classes=10
input_shape = (28, 28)

# data resize
X_train = img_resize(input_shape, X_train)
X_test = img_resize(input_shape, X_test)
print('data resize ',X_train.shape, X_test.shape)

# 차원 확장
X_train = np.expand_dims(X_train, axis = -1)
X_test = np.expand_dims(X_test, axis = -1)
print('expand_dims ',X_train.shape, X_test.shape)

# 정답값 categorical로
Y_train = utils.to_categorical(Y_train, 10)
Y_test = utils.to_categorical(Y_test, 10)
print('to_categorical ',X_train.shape, X_test.shape)

print('전처리 작업 완료')

--images are being resizing--
--image resize complete--
--images are being resizing--
--image resize complete--
data resize  (60000, 28, 28) (10000, 28, 28)
expand_dims  (60000, 28, 28, 1) (10000, 28, 28, 1)
to_categorical  (60000, 28, 28, 1) (10000, 28, 28, 1)
전처리 작업 완료


In [7]:
# 모델 저장 경로
MODEL_SAVE_FOLDER_PATH = 'model/'

# Lenet 모델 만듬
model = Sequential()

model.add(Conv2D(input_shape = (input_shape[0], input_shape[1], 1), filters = 4, kernel_size = (5, 5), strides=(1, 1), padding = 'same', name = 'conv1'))
model.add(Activation('tanh'))
model.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), name = 'avgpool1'))

model.add(Conv2D(filters = 12, kernel_size = (5, 5), strides=(1, 1), padding = 'valid', name = 'conv2'))
model.add(Activation('tanh'))
model.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), name = 'avgpool2'))

model.add(Flatten(name = 'flatten'))

model.add(Dense(classes, activation = 'softmax',  name = 'predictions'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'SGD', metrics = ['accuracy'])

history = model.fit(X_train, Y_train,
                    validation_data = (X_test, Y_test),
                    epochs = 1, batch_size = 1000, verbose = 1)

# 모델 save .h5 파일
model.save(MODEL_SAVE_FOLDER_PATH + "lenet-1.h5")
print('저장완료')

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
10000/10000 [==============================] - 2s 180us/step
prediction_acc:  0.4796000123023987
prediction_loss:  1.877234927368164
저장완료


In [10]:
from keras.models import load_model
from keras2onnx import convert_keras
model = load_model('model/lenet-1.h5')

# keras model -> onnx 로 변환
onx = convert_keras(model, 'lenet-1.onnx')

with open("model/lenet-1.onnx", "wb") as f:
    f.write(onx.SerializeToString())

print('onnx로 변환 완료')

onnx로 변환 완료
